In [ ]:
%load_ext nb_black

%load_ext autoreload
%autoreload 2

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import pandas as pd
from local_lib import utils, train
from pathlib import Path
import random
import shutil

In [ ]:
labels_pt = "../data/processed/labels_v0.1.0.csv"
source_image_dir = Path('../data/processed/labeling/images')
train_image_dir = Path('../data/processed/tmp_training')
shutil.rmtree(train_image_dir)
# TODO: figure out why 8 had to be substracted
max_images = 10_000 - 8

In [ ]:
labels = pd.read_csv(labels_pt)
remainder = max_images - len(labels)

# labels = labels.query("org_source != 'label_book'")

In [ ]:
_ = utils.copy_group_to_dir(df=labels, 
                        source_dir=source_image_dir, 
                        dest_dir=train_image_dir, 
                        group= ["org_source", "symbol"]
                       )
assert len(_) == 0

In [ ]:
labels_filtered = labels.query("suggestion != 'remove' and org_source != 'val'")

In [ ]:
images_2_aug = labels_filtered.query("confusing == 'none' and org_source != 'val'").copy()

In [ ]:
images_2_aug["aug_amount"] = 0

images_per_group = remainder // 10

symbol_counts = images_2_aug.symbol.value_counts()

for row in symbol_counts.to_frame().reset_index().itertuples():
    
    symbol_sample_min = int(images_per_group/row.symbol)
    sample_amount = [symbol_sample_min] * row.symbol
    sample_remainder = images_per_group - sum(sample_amount)
    if sample_remainder > 0:
        sample_amount[:sample_remainder] = [symbol_sample_min + 1] * sample_remainder
    random.shuffle(sample_amount)
    images_2_aug.loc[images_2_aug.symbol == row.index, "aug_amount"] = sample_amount

assert images_2_aug.aug_amount.sum() <= remainder

In [ ]:
import imgaug.augmenters as iaa
seq = iaa.Sequential([
    iaa.Crop(px=(1, 16), keep_size=True),
    iaa.Fliplr(0.5),
    iaa.GaussianBlur(sigma=(0, 3.0))
])

In [ ]:
utils.apply_aug(images_2_aug, source_image_dir, train_image_dir, seq, ["org_source", "symbol"])

In [ ]:
train.train_model(train_image_dir, "../data/raw/label_book", short_run=True)